## LangChain Expression Language (LCEL)

### Without LCEL (deprecated)

In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [3]:
prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

In [4]:
out_parser = StrOutputParser()

In [5]:
chain = LLMChain(prompt=prompt, llm=llm, output_parser=out_parser)

C:\Users\BEU7CA\AppData\Local\Temp\ipykernel_44708\2506595107.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm, output_parser=out_parser)


In [6]:
result = chain.invoke("retrieval augmented generation")
result

{'topic': 'retrieval augmented generation',
 'text': "### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge bases or documents during the generation process, thereby improving the relevance and accuracy of the generated content.\n\n#### Key Components\n1. **Retrieval Mechanism**: RAG employs a retrieval system to fetch relevant documents or pieces of information from a large corpus. This can be achieved using various techniques, including traditional keyword-based search, semantic search, or more advanced neural retrieval methods.\n\n2. **Generative Model**: After retrieving relevant information, RAG utilizes a generative model, typically based on transformer architectures (like BERT or GPT), to synthesize responses. Th

### LCEL

In [7]:
lcel_chain = prompt | llm | out_parser
# the output from prompt goes to llm and its output goes to out_parser
# prompt -> llm -> out_parser
# whatever the left side outputs will be fed into the right side

In [8]:
lcel_chain.invoke({"topic": "retrieval augmented generation"})

"### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.\n\n#### Concept Overview\nRAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:\n\n1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It often employs techniques such as dense vector representations and similarity search to identify the most pertinent data.\n\n2. **Generation Component**: Once the relevant information is retrieved, the generative model (often based on architectures like Transformers) use

#### How does the Pipe Operator Work?

Use the '|' operator \
Whatever the left side outputs will be fed into the right side

In [9]:
class Runnable:
    
    def __init__(self, func):
        self.func = func
    
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [10]:
def add_five(x):
    return x+5

def sub_five(x):
    return x-5

def mul_five(x):
    return x*5

In [11]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_five)
mul_five_runnable = Runnable(mul_five)

In [12]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

In [13]:
chain.invoke(3)

15

In [14]:
chain_pipe = add_five_runnable | sub_five_runnable | mul_five_runnable

In [15]:
chain_pipe.invoke(3)

15

### LCEL Runnable Lambda

In [16]:
from langchain_core.runnables import RunnableLambda

The Runnable above is not from LangChain \
to use the LangChain Runnable we use RunnableLambda

In [17]:
add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_five)
mul_five_runnable = RunnableLambda(mul_five)

In [18]:
chain_runnable = add_five_runnable | sub_five_runnable | mul_five_runnable

In [19]:
chain_runnable.invoke(3)

15

Lets try something more testing

In [20]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [21]:
chain = prompt | llm | out_parser

In [22]:
result = chain.invoke("AI")
result

'### Report on Artificial Intelligence (AI)\n\n#### Introduction\nArtificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies, including machine learning, natural language processing, robotics, and computer vision. AI has rapidly evolved over the past few decades, transforming industries and everyday life.\n\n#### Current State of AI\nAs of 2023, AI technologies are integrated into numerous applications, including:\n\n1. **Healthcare**: AI is used for diagnostics, personalized medicine, and predictive analytics. Machine learning algorithms analyze medical data to identify patterns and assist in treatment plans.\n\n2. **Finance**: AI algorithms are employed for fraud detection, algorithmic trading, and risk management. They analyze vast amounts of data to make real-time decisions.\n\n3. **Transportation**: Autonomous vehicles utilize AI for navigation and safety. AI systems pr

In [23]:
from IPython.display import display, Markdown

In [24]:
display(Markdown(result))

### Report on Artificial Intelligence (AI)

#### Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies, including machine learning, natural language processing, robotics, and computer vision. AI has rapidly evolved over the past few decades, transforming industries and everyday life.

#### Current State of AI
As of 2023, AI technologies are integrated into numerous applications, including:

1. **Healthcare**: AI is used for diagnostics, personalized medicine, and predictive analytics. Machine learning algorithms analyze medical data to identify patterns and assist in treatment plans.

2. **Finance**: AI algorithms are employed for fraud detection, algorithmic trading, and risk management. They analyze vast amounts of data to make real-time decisions.

3. **Transportation**: Autonomous vehicles utilize AI for navigation and safety. AI systems process data from sensors to make driving decisions.

4. **Customer Service**: Chatbots and virtual assistants powered by AI provide 24/7 customer support, handling inquiries and resolving issues efficiently.

5. **Manufacturing**: AI optimizes supply chains, predicts maintenance needs, and enhances production processes through automation.

#### Challenges and Ethical Considerations
Despite its benefits, AI poses several challenges:

- **Bias and Fairness**: AI systems can perpetuate existing biases present in training data, leading to unfair outcomes in areas like hiring and law enforcement.

- **Privacy Concerns**: The use of AI in data collection raises significant privacy issues, as personal information can be misused or inadequately protected.

- **Job Displacement**: Automation driven by AI may lead to job losses in certain sectors, necessitating workforce retraining and adaptation.

- **Accountability**: Determining responsibility for decisions made by AI systems, especially in critical areas like healthcare and autonomous driving, remains a complex issue.

#### Future Trends
The future of AI is promising, with several trends expected to shape its development:

- **Explainable AI**: There is a growing demand for AI systems that can provide transparent and understandable reasoning behind their decisions.

- **AI in Edge Computing**: As IoT devices proliferate, AI will increasingly be deployed at the edge, enabling real-time data processing and decision-making.

- **Collaborative AI**: Human-AI collaboration will become more prevalent, with AI augmenting human capabilities rather than replacing them.

- **Regulation and Governance**: As AI technologies advance, regulatory frameworks will likely evolve to address ethical concerns and ensure responsible use.

#### Conclusion
AI is a transformative technology with the potential to significantly impact various sectors. While it offers numerous benefits, it also presents challenges that require careful consideration and proactive management. As AI continues to evolve, fostering a balance between innovation and ethical responsibility will be crucial for its sustainable development.

In [25]:
old_word = "AI"
new_word = "skynet"

def replace_word(x):
    return x.replace(old_word, new_word)

In [26]:
# To make our own functions Runnable, it needs to be SINGLE arguments

In [27]:
replace_word_runnable = RunnableLambda(replace_word)

In [28]:
chain = prompt | llm | out_parser | replace_word

In [29]:
result = chain.invoke("AI")

In [30]:
display(Markdown(result))

### Report on Artificial Intelligence (skynet)

#### Introduction
Artificial Intelligence (skynet) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. skynet has rapidly evolved over the past few decades, becoming an integral part of various industries and daily life.

#### Current State of skynet
As of 2023, skynet technologies are being utilized across multiple sectors, including healthcare, finance, transportation, and entertainment. Key advancements include:

1. **Machine Learning (ML)**: Algorithms that enable computers to learn from and make predictions based on data. Deep learning, a subset of ML, has led to breakthroughs in image and speech recognition.

2. **Natural Language Processing (NLP)**: Technologies that allow machines to understand and respond to human language. Applications include chatbots, virtual assistants, and language translation services.

3. **Computer Vision**: The ability of machines to interpret and make decisions based on visual data. This technology is widely used in facial recognition, autonomous vehicles, and medical imaging.

4. **Robotics**: skynet-driven robots are increasingly used in manufacturing, logistics, and even healthcare, performing tasks ranging from assembly line work to surgery.

#### Benefits of skynet
- **Efficiency and Productivity**: skynet can automate repetitive tasks, allowing human workers to focus on more complex and creative endeavors.
- **Data Analysis**: skynet systems can analyze vast amounts of data quickly, providing insights that can drive business decisions and improve outcomes in fields like healthcare.
- **Personalization**: skynet enables personalized experiences in marketing, entertainment, and customer service, enhancing user satisfaction.

#### Challenges and Ethical Considerations
Despite its benefits, skynet also presents several challenges:

1. **Job Displacement**: Automation may lead to job losses in certain sectors, raising concerns about the future of work.
2. **Bias and Fairness**: skynet systems can perpetuate existing biases present in training data, leading to unfair outcomes in areas like hiring and law enforcement.
3. **Privacy Concerns**: The use of skynet in surveillance and data collection raises significant privacy issues, necessitating robust regulations.
4. **Accountability**: As skynet systems make more decisions, determining accountability for errors or harmful outcomes becomes increasingly complex.

#### Future Outlook
The future of skynet is promising, with ongoing research aimed at creating more advanced, ethical, and transparent systems. Emerging trends include:

- **Explainable skynet (Xskynet)**: Developing skynet systems that can explain their reasoning and decision-making processes to users.
- **skynet in Climate Change**: Leveraging skynet to model climate scenarios, optimize energy use, and develop sustainable practices.
- **Human-skynet Collaboration**: Fostering environments where humans and skynet work together, enhancing productivity and creativity.

#### Conclusion
Artificial Intelligence is transforming industries and society at large, offering significant benefits while also posing challenges that require careful consideration. As technology continues to evolve, it is crucial to address ethical concerns and ensure that skynet development aligns with societal values and needs. The future of skynet holds great potential, and its responsible integration into daily life will be key to maximizing its benefits.

### LCEL RunnableParallel and RunnablePassthrough

- RunnableParallel -> allows us to run multiple Runnable instance in parallel. Acting almost as a y-fork in the chain
- RunnablePassthrough -> allow us to pass through a variable to the next Runnable without modification

In [31]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [32]:
embedding = OpenAIEmbeddings()

vecstore_a = DocArrayInMemorySearch.from_texts(
    [
        "half the info is here",
        "DeepSeek-v3 was release in December 2024"
    ],
    embedding=embedding
)

vecstore_b = DocArrayInMemorySearch.from_texts(
    [
        "the other half of the info is here",
        "the DeepSeek-v3 LLM is a mixture of experts model with 671B parameters"
    ],
    embedding=embedding
)

c:\Users\BEU7CA\Documents\PersonalData\langchain-masterclass\.venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [33]:
prompt_str = """"
    Using the context provided, answer the user's question.

    Context:
    {context_a}
    {context_b}
"""

In [34]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [35]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt_str),
    HumanMessagePromptTemplate.from_template("{question}")
])

In [36]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [37]:
retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

# RunnableParallel -> Y-fork
# RunnablePassthrough -> pass through the var to the next Runnable without modifying it 
retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b, "question": RunnablePassthrough()
    }
)

In [38]:
chain = retrieval | prompt | llm | out_parser

In [39]:
chain.invoke("What architecture does the model DeepSeek release in december use?")

'The DeepSeek-v3 model, released in December 2024, is a mixture of experts model with 671 billion parameters.'